In [224]:
import pandas as pd
import os
os.chdir(r"C:\Users\chris\Documents\Transgola\Clients\PROJECTS\2022\444240122_TM_HS\Orignal\MemoQ") 

# choose language pairs
source = "English"
target = "Portuguese"

df = pd.read_excel(r"MemoQOut.xlsx", names=[source, target])

# make sure everything (dates, numbers etc.) are strings.
df = df.astype('str')

# remove white spaces at begining and end of string
df[source] = df[source].apply(lambda x: x.strip())

# create normalised column to filter on (lower)
df[f'{source}_2'] = df[source].str.lower().str.rstrip(":")

# subset performs operation on specified columns only (normalised column)
df.drop_duplicates(subset=[f'{source}_2'], inplace=True, keep="first")
df.drop(columns = f'{source}_2', inplace = True)
df.reset_index(inplace = True, drop = True)


# filter out unwanted tags


df = (
    df
    #.replace('nan','')
    .replace(r'\[[0-9+]\]?','', regex = True)
    .replace(r'\[[0-9+]\}?','', regex = True)
    .replace(r'\{[0-9+]\]?','', regex = True)
    .replace(r'\}\}?','', regex = True)
    .replace(r'\]?|\[?','', regex = True)
    .replace("•", "", regex = True)
    .replace(r'\S+@\S+', "" , regex = True) # email addresses
    .replace(r"(Table|Figure) [0-9]+:", "", regex = True) 
    .replace("", "EmptyCell")
    
)

# create list containing pre-filtered elements
preFiltered = df[source].tolist()

# filter cells
df = df[(df[source].str.contains("EmptyCell")==False)]
df = df[df[source].str.contains('[A-Za-z]')] 
df = df[df[source].str.len()>1]

# filter out strings ending with numbers but keep anything relivant
df = df[(~df[source].str.endswith(("0", "1", "2", "3", "4", "5", "6", "7", "8", "9"))) 
        & (~df[source].str.startswith(("Website", "E-mail", "http:", "www.")))
        | (df[source].str.endswith(("Cleaning) *1", "SARS-Cov-2")))]

# create list containing post-filtered elements
postFiltered = df[source].tolist()

# output cleaned (tags), dups removed, filtered version of MemoQ output
df.to_excel('memoQOutClean.xlsx')

# set character length for split
splitThreshold = 90

#define splits
forQuill =df[(df[source].str.len()>=splitThreshold) | (df[source].str.contains(",")) | (df[source].str.contains("\?"))]
forQuill =forQuill[(forQuill[target] == 'nan')]
forQuill.to_excel('forQuill.xlsx', index=False)

forDeepL =df[(df[source].str.len()<splitThreshold) & (~df[source].str.contains("\?")) & (~df[source].str.contains(","))]
forDeepL =forDeepL[(forDeepL[target] == 'nan')]
forDeepL.to_excel('forDeepL.xlsx', index=False)

# check that the cleaned output has the same amount of rows as the splits combined

if len(df[source]) == len(forQuill[target]) + len(forDeepL[source]):
    print("MemoQ master and DeepL + Quill splits are of equal length!\n")
else:
    print("MemoQ master and DeepL + Quill splits are NOT of equal length!\n")

print(f"{len(preFiltered)} = Pre-filtered length")
print(f"{len(postFiltered)} = Post-filtered length")
print("\n")
print("Removed during filtering\n")

results = set(preFiltered) - set(postFiltered)
for r in sorted(results,  key=len, reverse=True):
    print(r)

MemoQ master and DeepL + Quill splits are of equal length!

917 = Pre-filtered length
790 = Post-filtered length


Removed during filtering

(+244) 226 434 549 / 927 442 844 / 912 034 779
Telephone: (+244) 923 595 093 / 912 205 979
Telephone: (+244) 222 704 400 /222 704 401
http://www.facebook.com/holisticos.angola
9F KDX Toyosu Grand Square 1-7-12
(+244) 222 704 400 /222 704 401
Website: http://www.rnt.co.ao
http://www.holisticos.co.ao
Telephone: (+813) 6372 5183
Waste Management Plan4701
Waste Management Plan5201
Waste Management Plan4201
Waste Management Plan4901
Waste Management Plan1601
Waste Management Plan5401
Waste Management Plan5101
Website: www.tepsco.co.jp
Waste Management Plan5001
Waste Management Plan201
http://www.tepsco.co.jp
Table of Contentsii01
www.holisticos.co.ao
Rua 60, Casa No. 559
http://www.rnt.co.ao
2022-01-19 00:00:00
2021-12-13 00:00:00
(+244) 226 434 549
(+244) 927 442 844
(+244) 912 034 779
+244 923 41 01 86
1502/150202/03
4799693215
5401156421
EmptyCell
1

In [150]:
from collections import OrderedDict
import os
import pandas as pd


# forQuillDone & forDeepLDone are the translated versions of the splits (translation pairs)


os.chdir(r"C:\Users\chris\Documents\Transgola\Clients\PROJECTS\2022\437030122_TM_HS\Translation\MemoQ Out") 

quilled = pd.read_excel('forQuillDone.xlsx',  names=[source, target])
deepLed = pd.read_excel('forDeepLDone.xlsx',  names=[source, target])
memoQClean = pd.read_excel('memoQOutClean.xlsx')

frames = [quilled, deepLed]

conDf = pd.concat(frames, names=[source, target])
conDf.to_excel('conDF.xlsx')
conEn = conDf[target].tolist()
conPt = conDf[source].tolist()

memEn = memoQClean[target].tolist()
memPt = memoQClean[source].tolist()


conDict = OrderedDict(zip(conPt, conEn))
memoQCleanDict =  OrderedDict(zip(memPt, memEn))

for k in memoQCleanDict:
    if k in conDict:
        memoQCleanDict[k] = conDict[k]

In [21]:
preTrans = pd.DataFrame([memoQCleanDict]).T
preTrans = preTrans.reset_index()
preTrans.columns=[source, target]

In [22]:
preTrans.to_excel('preTrans.xlsx')

enList = preTrans[source].tolist()
ptList = preTrans[target].tolist()


textfile = open("EN.txt", "w")
for element in enList:
    textfile.write(element + "\n")
textfile.close()

textfile = open("PT.txt", "w")
for element in ptList:
    textfile.write(element + "\n")
textfile.close()


In [125]:
11867 - 13187


-1320

In [127]:
1320 * 12

15840